In [ ]:
%pip install pingouin

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import japanize_matplotlib

In [ ]:
def robust_anova_gameshowell(df, target_col, group_col):
    """
    等分散を仮定しない堅牢な分散分析（Welch ANOVA）と
    多重比較（Games-Howell法）を行う関数
    
    df: データフレーム
    target_col: 数値データのカラム名 (例: 'UWES_mean')
    group_col: 群分けのカラム名 (例: 'role')
    """
    
    # データのクリーニング（欠損値削除）
    # pingouinは欠損値に敏感なため、事前に削除したdfを作成します
    clean_df = df.dropna(subset=[target_col, group_col]).copy()
    
    # グループ数の確認
    groups = clean_df[group_col].unique()
    if len(groups) < 3:
        print(f"エラー: 3群以上のデータが必要です（現在は {len(groups)} 群: {groups}）。")
        print("2群の場合は Welchのt検定を使用してください。")
        return

    # --- 1. データの可視化 (箱ひげ図) ---
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=clean_df, x=group_col, y=target_col, hue=group_col, legend=False, palette="Set2")
    # データ点も重ねて表示（分布がわかりやすくなります）
    sns.stripplot(data=clean_df, x=group_col, y=target_col, color='black', alpha=0.3)
    plt.title(f'Comparison of {target_col} by {group_col} (n={len(clean_df)})')
    plt.show()

    print(f"\n{'='*60}")
    print(f" 分析対象: {target_col} / グループ: {group_col}")
    print(f"{'='*60}")

    # --- 2. WelchのANOVA (全体に差があるか？) ---
    # ddof1: 自由度1, p-unc: 補正なしp値, np2: 部分イータ二乗(効果量)
    aov = pg.welch_anova(dv=target_col, between=group_col, data=clean_df)
    
    print("\n【1. WelchのANOVA 結果】")
    print(aov)
    
    p_val_anova = aov['p-unc'][0]
    if p_val_anova < 0.05:
        print(f">> 判定: 有意差あり (p = {p_val_anova:.4f})")
        print("   → 群間のどこかに差があります。Games-Howell法の結果を確認してください。")
    else:
        print(f">> 判定: 有意差なし (p = {p_val_anova:.4f})")
        print("   → 全体として差は見られません。")

    print("-" * 60)

    # --- 3. Games-Howell法 (どのペアに差があるか？) ---
    # Welch ANOVAの結果に関わらず、詳細を見るために出力します
    print("\n【2. Games-Howell法 (多重比較) 結果】")
    
    gh = pg.pairwise_gameshowell(dv=target_col, between=group_col, data=clean_df)
    
    # 見やすいように必要な列だけ表示して整理
    # A, B: 比較している群
    # mean(A), mean(B): 各群の平均
    # diff: 差
    # pval: 補正後p値
    # hedges: 効果量(Hedges' g)
    display_cols = ['A', 'B', 'mean(A)', 'mean(B)', 'diff', 'pval', 'hedges']
    print(gh[display_cols].round(3))
    
    # 有意なペアの抽出表示
    sig_pairs = gh[gh['pval'] < 0.05]
    
    print("\n--- 有意差があったペアの要約 ---")
    if len(sig_pairs) > 0:
        for index, row in sig_pairs.iterrows():
            stars = "**" if row['pval'] < 0.01 else "*"
            print(f"・{row['A']} vs {row['B']} : p={row['pval']:.4f} {stars} (差: {row['diff']:.2f})")
    else:
        print("・有意な差があるペアはありませんでした。")

In [ ]:
if __name__ == "__main__":
    df = pd.read_excel('AdvanceData.xlsx')

    # 関数実行
    robust_anova_gameshowell(df, 'IADL', 'age')